In [1]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply, Flatten, BatchNormalization
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
from datetime import datetime

stock_name = 'AAPL'
now = datetime.now()
todays_date = now.strftime("%Y-%m-%d")

def train_and_save_model(stock_name):
    data = yf.download(stock_name, start='2020-01-01', end=todays_date)
    #data = yf.download('AAPL', period='3mo', interval='1d')
    print(data.tail())

    data.isnull().sum() # if number were not 0 then
    data.fillna(method='ffill', inplace=True)
    scaler = MinMaxScaler(feature_range=(0,1))
    data_scaled = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    sequence_length = 60

    x = []
    y = []

    for i in range (sequence_length, len(data_scaled)):
        x.append(data_scaled[i - sequence_length: i, 0])
        y.append(data_scaled[i, 0])


    x_train, y_train = np.array(x), np.array(y)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    # number of units the number of neurons
    model.add(LSTM(units=50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    attention = AdditiveAttention(name='attention_weight')
    model.add(Permute((2,1)))
    model.add(Reshape((-1, 50)))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='loss', patience=10)

    model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='loss')

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5)

    tensorboard = TensorBoard(log_dir='.logs')
    csv_logger = CSVLogger('training_log.csv')

    callbacks_list = [early_stopping, model_checkpoint, reduce_lr, tensorboard, csv_logger]
    history = model.fit(x_train, y_train, epochs=100, batch_size=25, callbacks=callbacks_list)

    model.save(f'models/{stock_name}/{stock_name}.h5')

stock_names = ['TSLA', 'GOOG', 'MSFT', 'AAPL', '^GSPC']

for stock in stock_names:
    train_and_save_model(stock)

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-09-16  229.300003  229.960007  223.529999  226.779999  226.779999   
2024-09-17  229.449997  234.570007  226.550003  227.869995  227.869995   
2024-09-18  230.089996  235.679993  226.880005  227.199997  227.199997   
2024-09-19  234.000000  244.240005  232.130005  243.919998  243.919998   
2024-09-20  241.520004  243.990005  235.919998  238.250000  238.250000   

               Volume  
Date                   
2024-09-16   54323000  
2024-09-17   66761600  
2024-09-18   78010200  
2024-09-19  102694600  
2024-09-20   99879100  
Epoch 1/100



/var/folders/vz/hr6jrn1n0ggf1mxbfs500y4m0000gn/T/ipykernel_26418/3075348991.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/Users/brianerichsenfagundes/Desktop/6017/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 1.2799 - learning_rate: 0.0010
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.5860 - learning_rate: 0.0010
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.5129 - learning_rate: 0.0010
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5192 - learning_rate: 0.0010
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.4336 - learning_rate: 0.0010
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3856 - learning_rate: 0.0010
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3189 - learning_rate: 0.0010
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3829 - learning_rate: 0.0010
Epoch 9/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2560 - learning_rate: 0.0010
Epoch 10/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2373 - learning_rate: 0.0010
Epoch 11/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2664 - learning_rate: 0.0010
Epoch 12/

[*********************100%%**********************]  1 of 1 completed

...

46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 1.5068 - learning_rate: 0.0010
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5733 - learning_rate: 0.0010
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.5332 - learning_rate: 0.0010
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.4286 - learning_rate: 0.0010
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.6929 - learning_rate: 0.0010
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3129 - learning_rate: 0.0010
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2809 - learning_rate: 0.0010
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2880 - learning_rate: 0.0010
Epoch 9/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2396 - learning_rate: 0.0010
Epoch 10/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2214 - learning_rate: 0.0010
Epoch 11/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1837 - learning_rate: 0.0010
Epoch 12/

[*********************100%%**********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2024-09-16  5615.209961  5636.049805  5604.529785  5633.089844  5633.089844   
2024-09-17  5655.509766  5670.810059  5614.049805  5634.580078  5634.580078   
2024-09-18  5641.680176  5689.750000  5615.080078  5618.259766  5618.259766   
2024-09-19  5702.629883  5733.569824  5686.419922  5713.640137  5713.640137   
2024-09-20  5709.640137  5715.140137  5674.490234  5702.549805  5702.549805   

                Volume  
Date                    
2024-09-16  3437070000  
2024-09-17  3443600000  
2024-09-18  3691390000  
2024-09-19  4024530000  
2024-09-20  7867260000  
Epoch 1/100



/var/folders/vz/hr6jrn1n0ggf1mxbfs500y4m0000gn/T/ipykernel_26418/3075348991.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/Users/brianerichsenfagundes/Desktop/6017/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 1.1038 - learning_rate: 0.0010
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.7944 - learning_rate: 0.0010
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.4953 - learning_rate: 0.0010
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.4047 - learning_rate: 0.0010
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3734 - learning_rate: 0.0010
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3330 - learning_rate: 0.0010
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2558 - learning_rate: 0.0010
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2675 - learning_rate: 0.0010
Epoch 9/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2468 - learning_rate: 0.0010
Epoch 10/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.2068 - learning_rate: 0.0010
Epoch 11/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1984 - learning_rate: 0.0010
Epoch 12/